In [1]:
import numpy as np
import argparse
import time
import datetime
import datautils
from utils import init_dl_program,dict2class
from infots import InfoTS as MetaInfoTS
# from baseline import InfoTS as baseInfoTS
from models.augclass import *

In [2]:
all_augs = [jitter(), scaling(), time_warp(), window_slice(), window_warp(),cutout(),subsequence()]

paras = {
    'dataset':'ETTh1', #electricity
    'archive':'forecast_csv_univar',
    'gpu':0,
    'seed':42,
    'max_threads':12,
    'log_file':'forecast_csv',
    'eval':True,
    'batch_size':128,
    'lr':0.001,
    'beta':0.5,
    'repr_dims':320,
    'max_train_length':2048,
    'iters':4000,
    'epochs':400,
    'dropout':0.1,
    'split_number':8,
    'label_ratio':1.0,
    'meta_beta':0.1,
    'aug':None,
    'aug_p1':0.7,
    'meta_lr':0.03
}

parser = argparse.ArgumentParser()
args = dict2class(**paras)


device = init_dl_program(args.gpu, seed=args.seed, max_threads=args.max_threads)

if args.archive == 'forecast_csv':
    task_type = 'forecasting'
    data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = datautils.load_forecast_csv(args.dataset)
    train_data = data[:, train_slice]
elif args.archive == 'forecast_csv_univar':
    task_type = 'forecasting'
    data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = datautils.load_forecast_csv(args.dataset, univar=True)
    train_data = data[:, train_slice]

valid_dataset = (data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols)

if train_data.shape[0] == 1:
    train_slice_number = int(train_data.shape[1] / args.max_train_length)
    if train_slice_number < args.batch_size:
        args.batch_size = train_slice_number
else:
    if train_data.shape[0] < args.batch_size:
        args.batch_size = train_data.shape[0]
print("Arguments:", str(args))

config = dict(
    batch_size=args.batch_size,
    lr=args.lr,
    meta_lr = args.meta_lr,
    output_dims=args.repr_dims,
    max_train_length=args.max_train_length,
    input_dims=train_data.shape[-1],
    device=device,
    num_cls =  args.batch_size,
    dropout = args.dropout,
)

t = time.time()


Arguments: <utils.dict2class object at 0x7f2d2ac3d040>


/a/bear.cs.fiu.edu./disk/bear-d/users/dluo/Desktop/testGNN/InfoTS/datautils.py:146: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  dt.weekofyear.to_numpy(),


In [3]:
model = MetaInfoTS(
    aug_p1= args.aug_p1,
    eval_every_epoch =1,
    **config
)

In [4]:
res = model.fit(train_data,
     task_type = task_type,
     meta_beta=args.meta_beta,
     n_epochs=args.epochs,
     n_iters=args.iters,
     beta = args.beta,
     verbose=False,
     miverbose=True,
     split_number=args.split_number,
     valid_dataset = valid_dataset,
     train_labels= None
    )

v,f, mse, mae = res
mi_info = 'v %.5f ,f %.5f,mse %.5f  mae%.5f' % (v,f,mse[-1], mae[-1])

print(mi_info)

t = time.time() - t
print(f"\nTraining time: {datetime.timedelta(seconds=t)}\n")
print("Finished.")

train_data_label [[(tensor([[ 0.0000, -1.6613,  0.4913,  ...,  0.1344, -0.0335,  1.4606],
        [ 0.0000, -1.5169,  0.4913,  ...,  0.1344, -0.0335,  1.1615],
        [ 0.0000, -1.3724,  0.4913,  ...,  0.1344, -0.0335,  1.1615],
        ...,
        [ 0.0000,  1.3724, -0.5080,  ...,  0.7106,  0.8279, -0.2490],
        [ 0.0000,  1.5169, -0.5080,  ...,  0.7106,  0.8279, -0.2030],
        [ 0.0000,  1.6613, -0.5080,  ...,  0.7106,  0.8279, -0.2490]]),), (tensor(0, device='cuda:0'),)], [(tensor([[ 0.0000, -1.6613, -0.0083,  ...,  0.7106,  0.8279, -0.3640],
        [ 0.0000, -1.5169, -0.0083,  ...,  0.7106,  0.8279, -0.7396],
        [ 0.0000, -1.3724, -0.0083,  ...,  0.7106,  0.8279, -0.7626],
        ...,
        [ 0.0000,  1.3724, -1.0077,  ...,  1.5749,  1.6893, -1.0617],
        [ 0.0000,  1.5169, -1.0077,  ...,  1.5749,  1.6893, -0.9543],
        [ 0.0000,  1.6613, -1.0077,  ...,  1.5749,  1.6893, -0.9926]]),), (tensor(1, device='cuda:0'),)], [(tensor([[ 0.0000, -1.6613, -0.5080,  .

/aul/homes/dluo/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=4.74854e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


[2022-11-24 09:26:53] Intermediate result: 0.19142497797103258  (Index 0)
{24: {'norm': {'MSE': 0.040736637257200975, 'MAE': 0.1506883407138316}}}
epoch_time 206.82215690612793


/aul/homes/dluo/.local/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:212: LinAlgWarning: Ill-conditioned matrix (rcond=4.90116e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


[2022-11-24 09:26:55] Intermediate result: 0.19088859946156023  (Index 1)
{24: {'norm': {'MSE': 0.04048341726338092, 'MAE': 0.1504051821981793}}}
epoch_time 131.40344619750977


...

{24: {'norm': {'MSE': 0.03998618989707565, 'MAE': 0.1517401195978757}}}
epoch_time 121.42181396484375
[2022-11-24 09:31:46] Intermediate result: 0.19176607111381344  (Index 228)
{24: {'norm': {'MSE': 0.039999424704477184, 'MAE': 0.15176664640933626}}}
epoch_time 196.98071479797363
[2022-11-24 09:31:48] Intermediate result: 0.19180520242217916  (Index 229)
{24: {'norm': {'MSE': 0.04001252430249736, 'MAE': 0.1517926781196818}}}
epoch_time 122.21646308898926
[2022-11-24 09:31:49] Intermediate result: 0.19184313793904906  (Index 230)
{24: {'norm': {'MSE': 0.04002533383622731, 'MAE': 0.15181780410282175}}}
epoch_time 194.90742683410645
[2022-11-24 09:31:50] Intermediate result: 0.19188057817359672  (Index 231)
{24: {'norm': {'MSE': 0.04003802793063364, 'MAE': 0.15184255024296306}}}
epoch_time 122.10345268249512
[2022-11-24 09:31:51] Intermediate result: 0.19191696817832607  (Index 232)
{24: {'norm': {'MSE': 0.04005045441454849, 'MAE': 0.15186651376377758}}}
epoch_time 185.0581169128418
[202

{24: {'norm': {'MSE': 0.04059040334334551, 'MAE': 0.15295263919191204}}}
epoch_time 129.8542022705078
[2022-11-24 09:32:43] Intermediate result: 0.19359635443344422  (Index 272)
{24: {'norm': {'MSE': 0.040607249749759676, 'MAE': 0.15298910468368454}}}
epoch_time 216.01128578186035
[2022-11-24 09:32:44] Intermediate result: 0.1936498593042724  (Index 273)
{24: {'norm': {'MSE': 0.04062417170555138, 'MAE': 0.153025687598721}}}
epoch_time 138.6723518371582
[2022-11-24 09:32:45] Intermediate result: 0.19370324249361195  (Index 274)
{24: {'norm': {'MSE': 0.04064106757446903, 'MAE': 0.15306217491914292}}}
epoch_time 192.98458099365234
[2022-11-24 09:32:47] Intermediate result: 0.19375677884026246  (Index 275)
{24: {'norm': {'MSE': 0.040658046282265596, 'MAE': 0.15309873255799686}}}
epoch_time 135.63060760498047
[2022-11-24 09:32:48] Intermediate result: 0.1938119066274101  (Index 276)
{24: {'norm': {'MSE': 0.040675574335884944, 'MAE': 0.15313633229152515}}}
epoch_time 211.9736671447754
[2022-

{24: {'norm': {'MSE': 0.04129273116965588, 'MAE': 0.15441890594293084}}}
epoch_time 130.31339645385742
[2022-11-24 09:33:43] Intermediate result: 0.1957542443828064  (Index 316)
{24: {'norm': {'MSE': 0.04130640190139725, 'MAE': 0.15444784248140914}}}
epoch_time 183.69269371032715
[2022-11-24 09:33:44] Intermediate result: 0.19579726229161032  (Index 317)
{24: {'norm': {'MSE': 0.041320192132807826, 'MAE': 0.1544770701588025}}}
epoch_time 112.39790916442871
[2022-11-24 09:33:45] Intermediate result: 0.19584067857018686  (Index 318)
{24: {'norm': {'MSE': 0.04133409807730736, 'MAE': 0.1545065804928795}}}
epoch_time 207.8261375427246
[2022-11-24 09:33:47] Intermediate result: 0.19588439927855317  (Index 319)
{24: {'norm': {'MSE': 0.04134811284420846, 'MAE': 0.15453628643434472}}}
epoch_time 125.83041191101074
[2022-11-24 09:33:48] Intermediate result: 0.19592821153800757  (Index 320)
{24: {'norm': {'MSE': 0.04136212239648824, 'MAE': 0.15456608914151934}}}
epoch_time 193.72177124023438
[2022

{24: {'norm': {'MSE': 0.04184768406350048, 'MAE': 0.15550792168720215}}}
epoch_time 127.44784355163574
[2022-11-24 09:34:40] Intermediate result: 0.1974061182161577  (Index 360)
{24: {'norm': {'MSE': 0.04186526646779241, 'MAE': 0.1555408517483653}}}
epoch_time 203.22465896606445
[2022-11-24 09:34:41] Intermediate result: 0.19745747979965445  (Index 361)
{24: {'norm': {'MSE': 0.041883120556835615, 'MAE': 0.15557435924281884}}}
epoch_time 128.37958335876465
[2022-11-24 09:34:42] Intermediate result: 0.19750918362052028  (Index 362)
{24: {'norm': {'MSE': 0.041901111533015706, 'MAE': 0.15560807208750457}}}
epoch_time 192.9178237915039
[2022-11-24 09:34:44] Intermediate result: 0.19756137738216076  (Index 363)
{24: {'norm': {'MSE': 0.041919286540746885, 'MAE': 0.15564209084141387}}}
epoch_time 136.0476016998291
[2022-11-24 09:34:45] Intermediate result: 0.19761356526997836  (Index 364)
{24: {'norm': {'MSE': 0.04193745985496046, 'MAE': 0.1556761054150179}}}
epoch_time 201.71594619750977
[202

ValueError: not enough values to unpack (expected 4, got 2)